# IMPORT MODULES

In [1]:
import os
from io import StringIO
import pandas as pd
import chardet
import os
import numpy as np
import re
from datetime import date
from datetime import datetime, timedelta
from dateutil import parser

In [2]:
def fix_dates(date_str):
    if not isinstance(date_str, str):
        return pd.NaT  # Return a 'Not a Timestamp' for non-string inputs (likely NaN)
    try:
        # dateutil's parse function will automatically infer the date format
        parsed_date = parser.parse(date_str)
        # Convert to pandas Timestamp
        timestamp = pd.Timestamp(parsed_date)
        # If the time is 00:00:00 (i.e., it's a date only), add a time of 00:00:00
        if timestamp.time() == pd.Timestamp('00:00:00').time():
            timestamp = timestamp.replace(hour=0, minute=0, second=0)
        return timestamp
    except ValueError:
        return pd.NaT  # Return a 'Not a Timestamp' for unparseable formats


xpert_files = pd.read_csv("E:\\CHPR_ORG\Bamenda Center for Health Promotion and Research\\INSPIRE TB Cameroon - INSPIRE TB PROJECT\\DO NOT MODIFY\\prospective_xpert_export_files.csv")

# extract the folder name
xpert_files['lab'] =  xpert_files['folder_path'].str.split('/').str[2]
# filter  out non csv files
xpert_files = xpert_files[xpert_files['file'].str.contains('.csv', case = False)]

# drop the folder path column
xpert_files.drop(columns = ['folder_path'], inplace = True)
xpert_files['start_date'] = xpert_files['start_date'].apply(fix_dates)
xpert_files['end_date'] = xpert_files['end_date'].apply(fix_dates)

# DEFINE THE LANGUAGE TRANSLATION DICTIONARY

In [3]:
language_map = {
	'INFORMATIONS SUR LE TEST' : 'ASSAY INFORMATION',
	"Paramètres d'analyse" : 'Analysis Settings',
	"TABLEAU DE RÉSULTATS" : 'RESULT TABLE',
	'Indice de cartouche' : "Cartridge Index",
	'Historique'  : 'History',
	"Résultat de l\'analyte" : 'Analyte Result',
	'Détail' : 'Detail',
	'Pics de fusion' :  'Melt Peaks',
	'Erreurs' : 'Errors',
	'Messages' : 'Messages',
	'Nom du module' : 'Module Name',
	'N° de série du module' : 'Module S/N',
	'Numéro de série de la cartouche' : 'Cartridge S/N', 
	"N° de série de l\'instrument": 'Instrument S/N', 
	'Version du logiciel' : 'S/W Version',
	'N° du lot' : 'Reagent Lot ID',
    'Date d\'expiration': 'Expiration Date',
	'Heure de lancement': 'Start Time',
	'Heure de fin' : 'End Time',
    "État de l\'erreur"  : 'Error Status',
	'État' : 'Status',
	'Utilisateur' :  'User',
	'Effectué' : 'Done',
	'Incomplet' : 'Incomplete',
	'Interrompu' : 'Done',
	
	"N° Id de l'échantillon" : 'Sample ID',
	'N° Id du patient' : 'Patient ID', 
	'Test' : 'Assay',
	'Version du test' : 'Assay Version',
	'Type de test' : 'Assay Type', 
	'Type de test' : 'Test Type',
	"Type d\'échantillon" : 'Sample Type',
	'Remarques' : 'Notes',
	'Résultat du test' : 'Test Result', 
	'Avertissement du test' : 'Test Disclaimer',
	"Date d'exportation" : 'Exported Date',
	'Clause de non-responsabilité concernant le test': 'Test Disclaimer',
	
	'MTB NON DÉTECTÉ||' : 'MTB NOT DETECTED||',
	'MTB DÉTECTÉ ÉLEVÉ||RIF Resistance NON DÉTECTÉ' : 'MTB DETECTED HIGH||RIF Resistance NOT DETECTED',
	'MTB DÉTECTÉ BAS||RIF Resistance NON DÉTECTÉ' : 'MTB DETECTED LOW||RIF Resistance NOT DETECTED',
	'MTB DÉTECTÉ MOYEN||RIF Resistance NON DÉTECTÉ' : 'MTB DETECTED MEDIUM||RIF Resistance NOT DETECTED',
	'MTB Trace DÉTECTÉ|RIF Resistance INDÉTERMINÉ' : '|MTB Trace DETECTED|RIF Resistance INDETERMINATE',
	'MTB DÉTECTÉ ÉLEVÉ||RIF Resistance DÉTECTÉ' : 'MTB DETECTED HIGH||RIF Resistance DETECTED',
	'MTB DÉTECTÉ TRÈS BAS||RIF Resistance NON DÉTECTÉ' : 'MTB DETECTED VERY LOW||RIF Resistance NOT DETECTED',
	'MTB DÉTECTÉ TRÈS BAS||RIF Resistance DÉTECTÉ' : 'MTB DETECTED VERY LOW||RIF Resistance DETECTED',
	'HIV-1 NON DÉTECTÉ|' : 'HIV-1 NOT DETECTED|',
	'MTB DÉTECTÉ BAS||RIF Resistance DÉTECTÉ':'MTB DETECTED LOW||RIF Resistance DETECTED',
    
	'MTB DÉTECTÉ MOYEN||RIF Resistance INDÉTERMINÉ': 'MTB DETECTED MEDIUM||RIF Resistance INDETERMINATE',
	'MTB DÉTECTÉ TRÈS BAS||RIF Resistance INDÉTERMINÉ': 'MTB DETECTED VERY LOW||RIF Resistance INDETERMINATE',
	'MTB DÉTECTÉ ÉLEVÉ||RIF Resistance INDÉTERMINÉ': 'MTB DETECTED HIGH||RIF Resistance INDETERMINATE',
	'MTB DÉTECTÉ MOYEN||RIF Resistance DÉTECTÉ' : 'MTB DETECTED MEDIUM||RIF Resistance DETECTED',
	'MTB DÉTECTÉ BAS||RIF Resistance INDÉTERMINÉ' : 'MTB DETECTED LOW||RIF Resistance INDETERMINATE',



	'NON VALIDE' : 'INVALID',
    'ERREUR' : 'ERROR',
	'PAS DE RÉSULTAT' : 'NO RESULT',
    '|MTB Trace DETECTED|RIF Resistance INDETERMINATE' : 'MTB Trace DETECTED|RIF Resistance INDETERMINATE',
    '|MTB Trace DÉTECTÉ|RIF Resistance INDÉTERMINÉ': 'MTB Trace DETECTED|RIF Resistance INDETERMINATE',
	'LS ' : '',
	',LS ': '',
	' LS' : '',
	'LS' : '',
	'LS:' : '',
	'CRACHAT' : 'SPUTUM',
	"Nom de l'utilisateur du rapport" : 'Report User Name', 
	'Contrôle de sonde 1' : 'Prb Chk 1',
	'Contrôle de sonde 2' : 'Prb Chk 2',
	'Contrôle de sonde 3' : 'Prb Chk 3',
	'Résultat du contrôle de la sonde' : 'Probe Check Result',
	'Dérivée seconde  de la hauteur  du pic' : '2nd Deriv Peak Height',
	'Ajustement de courbe' : 'Curve Fit',
	'Température pic de fusion' : 'Melt Peak Temperature',
	'Hauteur pic de fusion' : 'Melt Peak Height',
	'Diagnostic in vitro' : 'In Vitro Diagnostic',
	"Échantillon" : 'Specimen',
	"RÉUSSITE" : 'PASS',
	"ÉCHEC" : 'FAIL',
	"Nom de l'analyte" : 'Analyte Name',
	'SO,' : 'NA,',
	"SO" : 'NA',
    "RÉUSSITE" : 'PASS',
    
}

# define the folder directory
base_directory = "E:\\CHPR_ORG\\Bamenda Center for Health Promotion and Research\\Data Management - XPERT EXPORT PROJECT\\Prospective Xpert Exports"


# PROCESS ENGLISH FILES

In [5]:
# Function to detect the encoding of a file
def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        return chardet.detect(f.read())['encoding']

# Updated function for extraction focusing on the Detail section
def refined_extraction_with_detail(file_path, columns_of_interest, detail_columns, encoding):
    within_result_table = False
    within_detail_section = False
    processed_columns = set()
    detail_columns_found = set()

    with open(file_path, encoding=encoding) as f:
        for line in f:
            line = line.strip()

            if "RESULT TABLE" in line:
                processed_columns = set()
                within_detail_section = False

            if line in ["<Insufficient privilege to access data>", "\ufeff"]:
                continue

            for header in metadata_sections + ["RESULT TABLE", "Detail"]:
                if header in line:
                    if header == "RESULT TABLE":
                        within_result_table = True
                        yield {}
                    elif header == "Detail":
                        within_detail_section = True
                        detail_columns_found.clear()
                    elif header in metadata_sections:
                        within_result_table = False
                    break

            if within_detail_section and line.startswith(tuple(detail_columns)) and "," in line:
                key, value = line.split(",", 1)
                key = key.strip()
                if key in detail_columns_found:
                    continue
                detail_columns_found.add(key)
                yield {key: value.strip()}

            elif within_result_table and not within_detail_section:
                if line.startswith(tuple(columns_of_interest)) and "," in line:
                    key, value = line.split(",", 1)
                    key = key.strip()
                    if key in processed_columns:
                        continue
                    processed_columns.add(key)
                    yield {key: value.strip()}

# Function to get all CSV files in a directory and its subdirectories
def get_all_csv_files(directory):
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

# Process file function
def process_file(file_path, columns_of_interest, detail_columns):
    try:
        encoding = detect_encoding(file_path)
        patient_data = {}
        for extracted_data in refined_extraction_with_detail(file_path, columns_of_interest, detail_columns, encoding):
            if not extracted_data:
                if patient_data:
                    patient_data["file"] = os.path.basename(file_path)
                    all_df_rows.append(patient_data)
                    patient_data = {}
            else:
                patient_data.update(extracted_data)
        if patient_data:
            patient_data["file"] = os.path.basename(file_path)
            all_df_rows.append(patient_data)
    except UnicodeDecodeError:
        print(f"UnicodeDecodeError encountered while processing file: {file_path}")

# List of metadata sections to ignore
metadata_sections = ["GeneXpert Dx System", "ASSAY INFORMATION", "Analysis Settings"]

# Define the folder directory
base_directory = "E:\\CHPR_ORG\\Bamenda Center for Health Promotion and Research\\Data Management - XPERT EXPORT PROJECT\\Prospective Xpert Exports"

# List of columns of interest
columns_of_interest = [
    "Cartridge S/N", "Module Name", "Module S/N", 
    "Instrument S/N", "S/W Version", "Reagent Lot ID", "Expiration Date", 
    "Start Time", "End Time", "Error Status", "Status", "User", 
    "Sample ID", "Patient ID", "Assay", "Assay Version", 
    "Test Type", "Sample Type", "Notes", "Test Result"
]

# Additional columns to be extracted from the Detail section
detail_columns = ["SPC", "IS1081-IS6110", "rpoB1", "rpoB2", "rpoB3", "rpoB4"]

# Initialize a list to store all extracted rows
all_df_rows = []

# Process all files in the directory and its subdirectories
csv_files = get_all_csv_files(base_directory)
for file_path in csv_files:
    process_file(file_path, columns_of_interest, detail_columns)

# Create the final DataFrame
dfe = pd.DataFrame(all_df_rows)
print(f"Data successfully extracted from {len(csv_files)} files, resulting in {dfe.shape[0]} rows and {dfe.shape[1]} columns.")


UnicodeDecodeError encountered while processing file: E:\CHPR_ORG\Bamenda Center for Health Promotion and Research\Data Management - XPERT EXPORT PROJECT\Prospective Xpert Exports\TBRL Bamenda\TBRL BDA- XPERT DATA 1-12-23 TO 4-12-23.csv
UnicodeDecodeError encountered while processing file: E:\CHPR_ORG\Bamenda Center for Health Promotion and Research\Data Management - XPERT EXPORT PROJECT\Prospective Xpert Exports\TBRL Bamenda\TBRL BDA-XPERT DATA 11-11-23 TO 13-11-23.csv
Data successfully extracted from 176 files, resulting in 6926 rows and 38 columns.


# PROCESS THE FRENCH FILES

In [6]:
import os
import pandas as pd
import chardet

# Function to detect the encoding of a file
def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        return chardet.detect(f.read())['encoding']

# Function for extraction focusing on the Detail section
def refined_extraction_with_detail(file_path, columns_of_interest, detail_columns, encoding):
    within_result_table = False
    within_detail_section = False
    processed_columns = set()
    detail_columns_found = set()

    with open(file_path, encoding=encoding) as f:
        for line in f:
            line = line.strip()

            if "TABLEAU DE RÉSULTATS" in line:
                processed_columns = set()
                within_detail_section = False

            if line in ["<Privilège insuffisant pour accéder aux données>", "\ufeff"]:
                continue

            for header in metadata_sections + ["TABLEAU DE RÉSULTATS"]:
                if header in line:
                    if header == "TABLEAU DE RÉSULTATS":
                        within_result_table = True
                        yield {}
                    elif header in metadata_sections:
                        within_result_table = False
                    break

            if "Détail" in line:
                within_detail_section = True
                detail_columns_found.clear()
                continue

            if within_detail_section and line.startswith(tuple(detail_columns)) and "," in line:
                key, value = line.split(",", 1)
                key = key.strip()
                if key in detail_columns_found:
                    continue
                detail_columns_found.add(key)
                yield {key: value.strip()}

            elif within_result_table and not within_detail_section:
                if line.startswith(tuple(columns_of_interest)) and "," in line:
                    key, value = line.split(",", 1)
                    key = key.strip()
                    if key in processed_columns:
                        continue
                    processed_columns.add(key)
                    yield {key: value.strip()}

# Function to get all CSV files in a directory and its subdirectories
def get_all_csv_files(directory):
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

# Process file function
def process_file(file_path, columns_of_interest, detail_columns):
    try:
        encoding = detect_encoding(file_path)
        patient_data = {}
        for extracted_data in refined_extraction_with_detail(file_path, columns_of_interest, detail_columns, encoding):
            if not extracted_data:
                if patient_data:
                    patient_data["file"] = os.path.basename(file_path)
                    all_df_rows.append(patient_data)
                    patient_data = {}
            else:
                patient_data.update(extracted_data)
        if patient_data:
            patient_data["file"] = os.path.basename(file_path)
            all_df_rows.append(patient_data)
    except UnicodeDecodeError:
        print(f"UnicodeDecodeError encountered while processing file: {file_path}")

# List of metadata sections to ignore
metadata_sections = ["GeneXpert Dx System", "INFORMATIONS SUR LE TEST", "Paramètres d'analyse"]

# Define the folder directory
base_directory = "E:\\CHPR_ORG\\Bamenda Center for Health Promotion and Research\\Data Management - XPERT EXPORT PROJECT\\Prospective Xpert Exports"

# List of columns of interest
columns_of_interest = ['Numéro de série de la cartouche', 'Nom du module', 'N° de série du module', "N° de série de l'instrument",
                        'Version du logiciel', 'N° du lot',  "Date d'expiration", 'Heure de lancement',  'Heure de fin',  "État de l'erreur",
                        'État', 'Utilisateur',  "N° Id de l'échantillon",  'N° Id du patient',  'Test',  'Version du test',  'Type de test',
                        "Type d'échantillon",  'Remarques',  'Résultat du test']

# Additional columns to be extracted from the Detail section
detail_columns = ["SPC", "IS1081-IS6110", "rpoB1", "rpoB2", "rpoB3", "rpoB4"]

# Initialize a list to store all extracted rows
all_df_rows = []

# Process all files in the directory and its subdirectories
csv_files = get_all_csv_files(base_directory)
for file_path in csv_files:
    process_file(file_path, columns_of_interest, detail_columns)

# Create the final DataFrame
dff = pd.DataFrame(all_df_rows)
print(f"Data successfully extracted from {len(csv_files)} files, resulting in {dff.shape[0]} rows and {dff.shape[1]} columns.")

# Assuming language_map is a dictionary for column renaming
dff.rename(columns=language_map, inplace=True)
dff.replace(language_map, inplace=True)



UnicodeDecodeError encountered while processing file: E:\CHPR_ORG\Bamenda Center for Health Promotion and Research\Data Management - XPERT EXPORT PROJECT\Prospective Xpert Exports\TBRL Bamenda\TBRL BDA- XPERT DATA 1-12-23 TO 4-12-23.csv
UnicodeDecodeError encountered while processing file: E:\CHPR_ORG\Bamenda Center for Health Promotion and Research\Data Management - XPERT EXPORT PROJECT\Prospective Xpert Exports\TBRL Bamenda\TBRL BDA-XPERT DATA 11-11-23 TO 13-11-23.csv
Data successfully extracted from 176 files, resulting in 3102 rows and 36 columns.


# COMBINE AND PROCESS XPERT FILES

### Define Lists, Dictionaries for replacement

#### Define list of Pool IDs in Patient ID to be Replaced in Sample ID

In [ ]:
patient_id_list = [
    "P17 1-1-2-2",
    "P15 5-5",
    "P16 6-6",
    "P11 1-1-2",
    "P13 3-3",
    "P14 4-4",
    "P12 6-6",
    "P18 7-8",
    "POO2- 2-2",
    "P0001-1-3-8",
    "POOL-3 8-1-2",
    'P688-1-2-3',
    'P14  4-4',
    "POOl-3 8-1-2",
    'P0081-9-2-1-2',
    'P-000041-88-89-90',
    'P000040-85-86-87',
    'P000030-55-56-57',
    'P000029-52-53-54',
    'P000028-49-50-51',
    'P000027-46-47-48',
    'P000026-43-44-45',
    'P000025-40-41-42',
    'P000024-37-38-39',
    'P000024-37-38-39',
    'P000024-37-38-39',
    'P0057- 7-6-7-5 F 99',
    'P0056- 8-0-5-8 F 99',
    'POO55 6-2-5-3 F 99',
    'POO54 0-8-7-5 F 99',
    'POO53 6-9-5-3 F 99',
    'POO52 5-2-4-6 F',
    'POO49 2-3-9-2 F 99',
    'POO48 4-6-0-1 F 99',
    'POO51- 1-3-5-8 F 99',
    'P0050- 3-6-7-8 F 99',
    'POOL44 12-18-19-20 F 99',
    'POOL44 12-18-19-20 F',
    'POOL43 30-31-32-33 F 99',
    'POOL34 02-03-04-05 F 99',
    'POOL42 2699-2700 F 99',
    'POO67- 3-4 F 99',
    'P0062- 5-6-7-8 F 99',
    'POO 65 6-6-8 F 99',
    'POO63- 6-5-8 F 99',
    'POO61 1-7-4 F 99',
    'POO60 4-2-3 F 99',
    'POOL37 3-7-4-5 F 99',
    'POOL 36 60-62-63-64 F 99',
    'POOL35 3-7-58-110 F 99',
    'POOL33 09-22-52-111 F 99',
    'POOL34 24-49-50-51 F 99',
    'POOL32 108-107 F 99',
    'POOL31 103-106 F 99',
    'POOL30 102-17-46 F 99',
    'POOL29 99-101 F 99',
    'POOL28 87-89 F 99',
    'POOL 27 2-84-85-86 F 99',
    'POOL25 69-72-81-83 F 99',
    'POOL24 81-83-72-69 F 99',
    'POOL23 12-82-48-67 F 99',
    'POOL22 51-11-13-70 F 99',
    'POOL21 75-80-77-71 F 99',
    'POOL20 65-43-42 F 99',
    'POOL5 2-2-1 F 99',
    'POOL Esther.David.Tchoupoulo F 9',
    'POOL19 68-74-66 F 99',
    'POOL18 78-76-73-75 F 99',
    'POOL17 5-7-47 F 99',
    'POOL16 43-44-45-46 F 99',
    'POOL15 8-10-36-41 F 99',
    'POOL14 27-33-34-38 F 99',
    'POOL13 32-35-37-38 F 99',
    'POOL12 24--41-31-50 F 99',
    'POOL12 24-31-41-50 F 99',
    'POOL11 25-26-28 F 99',
    'POOL11 25-26-28 F 99',
    'POOL11 25-26-28 F 99',
    'POOL10 21-23-39 F 99',
    'POOL9 1-6-20 F 99',
    'POOL8 4-18-17-3 F 99',
    'POOL7 11-13-14-16 F',
    'POOL6 12-15 F 99',
    'POO2 1-2-3-37 F 99',
    'P040-5-6-7',
    'P030-5-6-7',
    'P029-2-3-4',
    'P028-9-0-1',
    'P027-6-7-8',
    'P026-3-4-5',
    'P025-0-1-2',
    "p0058- 9-3-6-4 F 99",
    "P0006-7-8",
    "P0003-4-5",
    "P0022-0-3-1-7-7"
    


]

#### Dictionary of Pool IDs to be replaceed with the Correct Ones

In [ ]:
# dictionary of Pool IDs to repalce
pool_id_dict = {
    'P0057- 7-6-7-5 F 99' : 'P0057-7-6-7-5',
    'P0056- 8-0-5-8 F 99' : 'P0056-8-0-5-8',
    'POO55 6-2-5-3 F 99' : 'POO55-6-2-5-3',
    'POO54 0-8-7-5 F 99' : 'POO54-0-8-7-5',
    'POO53 6-9-5-3 F 99' : 'POO53-6-9-5-3',
    'POO52 5-2-4-6 F' : 'POO52-5-2-4-6',
    'POO49 2-3-9-2 F 99' : 'POO49-2-3-9-2',
    'POO48 4-6-0-1 F 99' : 'POO48-4-6-0-1',
    'POO51- 1-3-5-8 F 99' : 'POO51-1-3-5-8',
    'P0050- 3-6-7-8 F 99' : 'P0050-3-6-7-8',
    'POOL44 12-18-19-20 F 99' : 'POOL44-12-18-19-20',
    'POOL44 12-18-19-20 F' : 'POOL44-12-18-19-20',
    'POOL43 30-31-32-33 F 99' : 'POOL43-30-31-32-33',
    'POOL34 02-03-04-05 F 99' : 'POOL34-02-03-04-05',
    'POOL42 2699-2700 F 99' : 'POOL42-2699-2700',
    'POO67- 3-4 F 99' : 'POO67-3-4',
    'P0062- 5-6-7-8 F 99' : 'P0062-5-6-7-8',
    'POO 65 6-6-8 F 99' : 'POO-65-6-6-8',
    'POO63- 6-5-8 F 99' : 'POO63-6-5-8',
    'POO61 1-7-4 F 99' : 'POO61-1-7-4',
    'POO60 4-2-3 F 99' : 'POO60-4-2-3',
    'POOL37 3-7-4-5 F 99' : 'POOL37-3-7-4-5',
    'POOL 36 60-62-63-64 F 99' : 'POOL-36-60-62-63-64',
    'POOL35 3-7-58-110 F 99' : 'POOL35-3-7-58-110',
    'POOL33 09-22-52-111 F 99' : 'POOL33-09-22-52-111',
    'POOL34 24-49-50-51 F 99' : 'POOL34-24-49-50-51',
    'POOL32 108-107 F 99' : 'POOL32-108-107',
    'POOL31 103-106 F 99' : 'POOL31-103-106',
    'POOL30 102-17-46 F 99' : 'POOL30-102-17-46',
    'POOL29 99-101 F 99' : 'POOL29-99-101',
    'POOL28 87-89 F 99' : 'POOL28-87-89',
    'POOL 27 2-84-85-86 F 99' : 'POOL-27-2-84-85-86',
    'POOL25 69-72-81-83 F 99' : 'POOL25-69-72-81-83',
    'POOL24 81-83-72-69 F 99' : 'POOL24-81-83-72-69',
    'POOL23 12-82-48-67 F 99' : 'POOL23-12-82-48-67',
    'POOL22 51-11-13-70 F 99' : 'POOL22-51-11-13-70',
    'POOL21 75-80-77-71 F 99' : 'POOL21-75-80-77-71',
    'POOL20 65-43-42 F 99' : 'POOL20-65-43-42',
    'POOL5 2-2-1 F 99' : 'POOL5-2-2-1',
    'POOL Esther.David.Tchoupoulo F 9' : 'POOL23-33-34-35',
    'POOL19 68-74-66 F 99' : 'POOL19-68-74-66',
    'POOL18 78-76-73-75 F 99' : 'POOL18-78-76-73-75',
    'POOL17 5-7-47 F 99' : 'POOL17-5-7-47',
    'POOL16 43-44-45-46 F 99' : 'POOL16-43-44-45-46',
    'POOL15 8-10-36-41 F 99' : 'POOL15-8-10-36-41',
    'POOL14 27-33-34-38 F 99' : 'POOL14-27-33-34-38',
    'POOL13 32-35-37-38 F 99' : 'POOL13-32-35-37-38',
    'POOL12 24--41-31-50 F 99' : 'POOL12-24-41-31-50',
    'POOL12 24-31-41-50 F 99' : 'POOL12-24-31-41-50',
    'POOL11 25-26-28 F 99' : 'POOL11-25-26-28',
    'POOL11 25-26-28 F 99' : 'POOL11-25-26-28',
    'POOL11 25-26-28 F 99' : 'POOL11-25-26-28',
    'POOL10 21-23-39 F 99' : 'POOL10-21-23-39',
    'POOL9 1-6-20 F 99' : 'POOL9-1-6-20',
    'POOL8 4-18-17-3 F 99' : 'POOL8-4-18-17-3',
    'POOL7 11-13-14-16 F' : 'POOL7-11-13-14-16',
    'POOL6 12-15 F 99' : 'POOL6-12-15',
    'POO2 1-2-3-37 F 99' : 'POO2-1-2-3-37',
    'P-000041-88-89-90' : 'P000041-88-89-90',
    "4BRH-01749-NA-NEW-HOS-SPU" : "P000041-88-89-90-80-80-80-80",
    "6MBI-00301-AH" : "P000041-8-8-8-8",
    "p0058- 9-3-6-4 F 99" : "P0058-9-3-6-4",
    "1MOK000095" : "1MOK-00095",
    "1MOK000094" : "1MOK-00094",
    "1MOK0000180" : "1MOK-00180",
    '2' : 'P000041-88-89-72',
    "21" : "P000041-83-99-71",



    
}

#### Define a dictionary of Clean Notes to be replaced with the correct counterpart

In [ ]:

# define a dictionary to further clean the notes field
clean_notes_replacement_dict = {"KB.PP.SM.YC P1711-8-0-1-2 LS-120058-KB-2 LS-120060-PP-1 LS-120061-SM-1 LS-120062-YC-1": "LS-120058-KB-2 LS-120060-PP-1 LS-120061-SM-1 LS-120062-YC-1",
    "POOL NJ.AG.BA.BA P1710-4-5-9-9 LS-120054-NJ-2 LS-120055-AG-2 LS-120059-BA-1 LS-120059-BA-2" : "LS-120054-NJ-2 LS-120055-AG-2 LS-120059-BA-1 LS-120059-BA-2",
    "4BRH-00419-S 4BRH-00417-TG 4BRH-00354-4BRH-00420-OB": "4BRH-00419-S 4BRH-00417-TG 4BRH-00354 4BRH-00420-OB",
    "1 MOR-00039-A": "1MOR-00039-A",
    "2ESP-00234 2ESP-00235-2ESP-00237" : "2ESP-00234 2ESP-00235 2ESP-00237",
    "2ESP-00234 2ESP-00235-2ESP-00237" : "2ESP-00234 2ESP-00235 2ESP-00237",
    "KB.PP.SM.YC P1711-8-0-1-2 LS-120058-KB-2 LS-120060-PP-1 LS-120061-SM-1 LS-120062-YC-1" : "LS-120058-KB-2 LS-120060-PP-1 LS-120061-SM-1 LS-120062-YC-1",
    "POOL NJ.AG.BA.BA P1710-4-5-9-9 LS-120054-NJ-2 LS-120055-AG-2 LS-120059-BA-1 LS-120059-BA-2" : "LS-120054-NJ-2 LS-120055-AG-2 LS-120059-BA-1 LS-120059-BA-2",
    "4BRH-00419-S 4BRH-00417-TG 4BRH-00354-4BRH-00420-OB": "4BRH-00419-S 4BRH-00417-TG 4BRH-00354 4BRH-00420-OB",
    "101 10113-YM 10115-AB 10116-SS 10110-TE": "10113-YM 10115-AB 10116-SS 10110-TE",
    "00419-S 00417-TG 00354-00420-OB": "00419-S 00417-TG 00354 00420-OB",
    "4SJC-00002-JL 4SJC-00003- MA 4SJC-00004-ER 4SJC-00005-TC": "4SJC-00002-JL 4SJC-00003-MA 4SJC-00004-ER 4SJC-00005-TC",
    "126329-126330-AJ 126331-NR 126332-BB": "126329 126330-AJ 126331-NR 126332-BB",
    "6MBI-00442-6MMI-00443-BA": "6MBI-00442 6MMI-00443-BA",
    "6MBP-01805-MN 6MBP-01806-6MBP-01807-WZ 6MBP-01808-NA": "6MBP-01805-MN 6MBP-01806 6MBP-01807-WZ 6MBP-01808-NA",
    "6MBP-01221-6MBP-01222-MM 6MBP-01223-NN 6MBP-01224-ND" : "6MBP-01221 6MBP-01222-MM 6MBP-01223-NN 6MBP-01224-ND",
    "6MBI-00442-6MMI-00443-BA": "6MBI-00442 6MMI-00443-BA",
    "6MBP-01805-MN 6MBP-01806-6MBP-01807-WZ 6MBP-01808-NA": "6MBP-01805-MN 6MBP-01806 6MBP-01807-WZ 6MBP-01808-NA",
    "6MBP-02559-TC 6MBP-02561-6MBP-02562-NL 6MBP-02563-NE": "6MBP-02559-TC 6MBP-02561 6MBP-02562-NL 6MBP-02563-NE",
    "1DAR-00091-IM 1DJ-00661-1DJK-0034-FN 1DAR-00083-MA": "1DAR-00091-IM 1DJ-00661 1DJK-0034-FN 1DAR-00083-MA",
    "1DAR-00068-TYB1DAR-00067-DY 1DAR-00069-ST 1DAR-00070-S0 1FOU-00698-AB 1FOU-00635-OA 1FOU-00696-HB 1FOU-00700-MH": "1DAR-00068-TYB 1DAR-00067-DY 1DAR-00069-ST 1DAR-00070-S0 1FOU-00698-AB 1FOU-00635-OA 1FOU-00696-HB 1FOU-00700-MH",
    "1DJK-00569-ST 1DJK-00568-FA 1DJK-00567-DA 1DJK-00566- SI 1DJK-00575-LM": "1DJK-00569-ST 1DJK-00568-FA 1DJK-00567-DA 1DJK-00566-SI 1DJK-00575-LM",
    "62070-YS-SPU-1 62073-AC- SPU-1 62099-RS- SPU-1" :  "62070-YS-SPU-1 62073-AC-SPU-1 62099-RS-SPU-1",
    "HAOUA HAMADOU-320-HH-SPU-FOUN MAIRAOU BOUBA-321-MB-SPU-FOUN": "320-HH-SPU-FOUN 321-MB-SPU-FOUN",
    "11541-DM-SPU-1 L 11539-GM-SPU-1" : "11541-DM-SPU-1 11539-GM-SPU-1",
    "PS 10113-BE-SPU-1 PS10112-DB-SPU-1" : "10113-BE-SPU-1 10112-DB-SPU-1",
    "101111-AM-SPU- 1 101114-FM -SPU-1 101110 -AA-SPU-1" : "101111-AM-SPU-1 101114-FM-SPU-1 101110-AA-SPU-1",
    "11394-AS -SPU-1 11395-AS-SPU-1 11396-NL-SPU-1 11398-OH-SPU-1" : "11394-AS-SPU-1 11395-AS-SPU-1 11396-NL-SPU-1 11398-OH-SPU-1",
    "HAMADOU SOULEY A et B" :"",
    "61493-PS-1SPU-00001": "61493-PS-1 SPU-00001",
    "1 P0001-1DER-00033-BT 2 P0001-1DER-00034-GE 3 P0001-1DER-00035-BD" : "1DER-00033-BT 1DER-00034-GE 1DER-00035-BD",
    '1MOK-000088-VV-1MOK-000089-NN-1MOK-000090-CC' : '1MOK-00088-VV- 1MOK-00089-NN- 1MOK-00090-CC',
    '1MOK-000085-DD-1MOK-000086-CC-1MOK-000087-KC' : '1MOK-00085-DD- 1MOK-00086-CC- 1MOK-00087-KC',
    '1MOK-000055-HH-1MOK-000056-NL-1MOK-000057-NN' : '1MOK-00055-HH- 1MOK-00056-NL- 1MOK-00057-NN',
    '1MOK-000052-HB-1MOK-000053-VG-1MOK-000054-BD' : '1MOK-00052-HB- 1MOK-00053-VG- 1MOK-00054-BD',
    '1MOK-000049-FJ-1MOK-000050-HH-1MOK-000051-SA' : '1MOK-00049-FJ- 1MOK-00050-HH- 1MOK-00051-SA',
    '1MOK-000046-MM-1MOK-000047-FF-1MOK-0000-48-AA' : '1MOK-00046-MM- 1MOK-00047-FF- 1MOK-00048-AA',
    '1MOK-000043-ZM-1MOK-000044-HD-1MOK-000045-RH' : '1MOK-00043-ZM- 1MOK-00044-HD- 1MOK-00045-RH',
    '1MOK-000040-EM-1MOK-000041-FK-1MOK-000042-MM' : '1MOK-00040-EM- 1MOK-00041-FK- 1MOK-00042-MM',
    '1MOK-000037KB-1MOK-000038LJ-1MOK-000039HA' : '1MOK-00037KB- 1MOK-00038LJ- 1MOK-00039HA',
    '1MOK0000-34-KW-1MOK0000-35-LI-1MKL0000-36-GA' : '1MOK-00034-KW- 1MOK-00035-LI- 1MKL-00036-GA',
    '1MOK000-9-MT-1MOK000-10-WS-1MOK000-11-DG' : '1MOK-00009-MT- 1MOK-00010-WS- 1MOK-00011-DG',
    '1MOK000-15-AG-1MOK000-16-AO-1MOK000-17-YR' : '1MOK-00015-AG- 1MOK-00016-AO- 1MOK-00017-YR',
    '1MOK-000037KB-1MOK-000038LJ-1MOK-000039HA' : '1MOK-00037KB- 1MOK-00038LJ- 1MOK-00039HA',
    '1MOK000-15-AG-1MOK000-16-AO-1MOK000-17-YR' : '1MOK-00015-AG- 1MOK-00016-AO- 1MOK-00017-YR',
    '1MOK-000037KB-1MOK-000038LJ-1MOK-000039HA' : '1MOK-00037KB- 1MOK-00038LJ- 1MOK-00039HA',
    '1MOK000-15-AG-1MOK000-16-AO-1MOK000-17-YR' : '1MOK-00015-AG- 1MOK-00016-AO- 1MOK-00017-YR',
    "1MOK-00049FJ 1MOK-00050-HH-1MOK-00051-SA" : "1MOK-00049FJ 1MOK-00050-HH- 1MOK-00051-SA",
    "1MOK-00049FJ 1MOK-00050-HH-1MOK-00051-SA" : "1MOK-00049FJ 1MOK-00050-HH- 1MOK-00051-SA",



    '6MBI-00442-LS 6MMI-00443-BA' : '6MBI-00442-LS 6MBI-00443-BA',
    '6MBI-00442-LS 6MMI-00443-BA' : '6MBI-00442-LS 6MBI-00443-BA',
    'POO55-1YAG-00129NT-1YAG-00135GC-1YAG-00137KC-1YAG-00114HR' : '1YAG-00129NT- 1YAG-00135GC- 1YAG-00137KC- 1YAG-00114HR',
    'POO54-1YAG-00120HI-1YAG-00127DA-1YAG-00114HC-1YAG-00119VB' : '1YAG-00120HI- 1YAG-00127DA- 1YAG-00114HC- 1YAG-00119VB',
    'POO53-1YAG-00151-TN-1YAG-00149TD-1YAG-00152NA-1YAG-00150RO' : '1YAG-00151-TN- 1YAG-00149TD- 1YAG-00152NA- 1YAG-00150RO',
    'POO67-1YAG-00213AL-1YAG00214GA' : '1YAG-00213AL- 1YAG00214GA',
    'POO62-1DAN-00005GC-1DAN-00006NB-1DAN-00007MI-1DAN-00008HM' : '1DAN-0005GC- 1DAN-0006NB- 1DAN-0007MI- 1DAN-0008HM',
    'POO61-2801-BG-2807-MA-2804-OM' : '2801-BG- 2807-MA- 2804-OM',
    'POO60-1YAG-00162-FA-1YAG-00163-FB-1YAG-00164-GJ' : '1YAG-00162-FA- 1YAG-00163-FB- 1YAG-00164-GJ',
    "POO67-1YAG-00213AL-1YAG00214GA" : "1YAG-00213 1YAG-00214",
    "POO67-1YAG-00213AL-1YAG-00214GA" : "1YAG-00213 1YAG-00214",
    "POOL44-2712GORSOU" : np.nan,
    "61493-PS-1SPU-1" : np.nan,
    "CONTAMINATION CHECK INSTRUMENT A CONTAMINATION CHECK INSTRUMENT C SURFACE CONTAMINATION CHECK" : np.nan,
    "CON CHECK A CON CHECK C SURFACE CON CHECK" : np.nan,
    "5MUT-012 5MUT-012 5MUT-012 5MUT-01299-NB 5MUT-01300-MK 5MUT-01301-DM 5MUT-01302-AG" : "5MUT-01299-NB 5MUT-01300-MK 5MUT-01301-DM 5MUT-01302-AG",
    "2GNA-00023-DS 2GNA-00010-2GNA-00024-MC" : "2GNA-00023-DS 2GNA-00010 2GNA-00024-MC",
    "POOL43 2702-FADIMATOU-2703-LOGOSSAI-2704-FLEKKA-2705-YADI" : "2702 2703 2704 2705",
    "4TDH-00005-TJ 4TDH-0005MUT-01408-DG 5MUT-01409-TE" : "4TDH-00005-TJ 4TDH-00006 5MUT-01408-DG 5MUT-01409-TE",
    "4NHC-00055-AE 4NHC-00056-4NHC-00057-NH 4NHC-00058-NM" : "4NHC-00055-AE 4NHC-00056 4NHC-00057-NH 4NHC-00058-NM",
    "2DJA-00846 2DJA-00863 2DJA-00600 2DJA-899" : "2DJA-00846 2DJA-00863 2DJA-00600 2DJA-00899",
    "6MBP-02866MBP-02865-SS 6MBP-02866-TM 6MBP-02867-TS 6MBP-02868-NK" : "6MBP-02865-SS 6MBP-02866-TM 6MBP-02867-TS 6MBP-02868-NK",
    "POOL44-2712-GORSOU ETIENNE-2718-ZARA KOURANA-2719-MAIGONWA MARTINE-2720-MAHAMAT" : "2712 2718 2719 2720",
    "POO49 1YAG-00132LH 1YAG-00133MJ 1YAG-00139DG 1YAG-00142LC" : "1YAG-00132LH 1YAG-00133MJ 1YAG-00139DG 1YAG-00142LC",
    "POOL43 1CYA-00030AISSATOU GAMBO1YCA-00031AWIKOUA EUGENIE1CYA-00032ADAMOU SOUALEY1CYA-00033ZANNOUBA": "1CYA-00030 1CYA-00031 1CYA-00032 1CYA-00033",
    "Salomond paka MTB833 Wadja chikabou MTB827" : "MTB833 MTB827",
    "POO52 1YAG-00115ME 1YAG-001122GU 1YAG-00124NC1YAG-00126MD" : "1YAG-00115ME 1YAG-001122GU 1YAG-00124NC 1YAG-00126MD",
    "POOL42 2669-HAYATOU-2700-LITASSOU" : "2669 2700",
    "POO 65-1YAG-00166-MC-1CYA-00036-RO-1CYA-00038-DS": "1YAG-00166-MC 1CYA-00036-RO 1CYA-00038-DS",
    "KB.PP.SM.YC P1711-8-0-1-2 LS-120058-KB-2 LS-120060-PP-1 LS-120061-SM-1 LS-120062-YC-1" : "LS-120058-KB-2 LS-120060-PP-1 LS-120061-SM-1 LS-120062-YC-1",
    "POOL NJ.AG.BA.BA P1710-4-5-9-9 LS-120054-NJ-2 LS-120055-AG-2 LS-120059-BA-1 LS-120059-BA-2" : "LS-120054-NJ-2 LS-120055-AG-2 LS-120059-BA-1 LS-120059-BA-2",
    "KB-PP-SM-YC P1711-8-0-1-2 LS-120058-KB-2 LS-120060-PP-1 LS-120061-SM-1 LS-120062-YC-1" : "LS-120058-KB-2 LS-120060-PP-1 LS-120061-SM-1 LS-120062-YC-1",


}


#### qr_code_replacement_dict

In [ ]:
qr_code_replacement_dict = {

    '1D1R-00018-GH' : '1DAR-00018',
    '1DAR-00057-' : '1DAR-00057',
    '1DJO0-00001-OA' : '1DJK-00001',
    '1DJK-OO228' : '1DJK-00228',
    '1DJK-OO230' : '1DJK-00230',
    '1DJK-OO233' : '1DJK-00233',
    '1DJ-00650-VV' : '1DJK-00650',
    '1DJ-00661' : '1DJK-00661',
    '1K1L-00009-YO' : '1KAL-00009',
    '1KGAL-00073-VA' : '1KAL-00073',
    '1KGAL-00074-DH' : '1KAL-00074',
    '1KR-00003-HA' : '1KRG-00003',
    '1KR-00003-HA' : '1KRG-00003',
    '1MKL0000-58-WM' : '1MKL-00058',
    '1MKL0000-59-AD' : '1MKL-00059',
    '1MKL0000-60-VB' : '1MKL-00060',
    '1MKL0000-62-YD' : '1MKL-00062',
    '1MKL0000-63-DR' : '1MKL-00063',
    '1MKL0000-64-AA' : '1MKL-00064',
    '1MKL0000-65-AD' : '1MKL-00065',
    '1MKL0000-66-MA' : '1MKL-00066',
    '1MKL0000-67-DA' : '1MKL-00067',
    '1MKL0000-68-FA' : '1MKL-00068',
    '1MKL0000-69-AB' : '1MKL-00069',
    '1MKL0000-70-AD' : '1MKL-00070',
    '1MKL0000-71-TA' : '1MKL-00071',
    '1MKL0000-72-BD' : '1MKL-00072',
    '1MKL0000-73-FJ' : '1MKL-00073',
    '1MKL0000-74-GB' : '1MKL-00074',
    '1MKL0000-75NT' : '1MKL-00075',
    '1MKL0000-76-MR' : '1MKL-00076',
    '1MKL0000-77-AA' : '1MKL-00077',
    '1MKL0000-78-AB' : '1MKL-00078',
    '1MKL0000-79-DG' : '1MKL-00079',
    '1MKL0000-80-DW' : '1MKL-00080',
    '1MKL0000-81-JA' : '1MKL-00081',
    '1MOK0009-MT' : '1MOK-00009',
    '1MOK0000-24-MM' : '1MOK-00024',
    '1MOK0000-25-WD' : '1MOK-00025',
    '1MOK0000-26-MS' : '1MOK-00026',
    '1MOK0000-31-D0' : '1MOK-00031',
    '1MOK0000-32-DC' : '1MOK-00032',
    '1MOK0000-33-GK' : '1MOK-00033',
    '1MOK0043-ZM' : '1MOK-00043',
    '1MOK0000-73-FJ' : '1MOK-00073',
    '1MOK0000-74-GB' : '1MOK-00074',
    '1MOK0000-75-NT' : '1MOK-00075',
    '1MOK0000-79-DG' : '1MOK-00079',
    '1MOK0000-80-DW' : '1MOK-00080',
    '1MOK0000-81-JA' : '1MOK-00081',
    '1MOK0000-82-AT' : '1MOK-00082',
    '1MOK0000-82-AT' : '1MOK-00082',
    '1MOK0000-83-KT' : '1MOK-00083',
    '1MOK0000-83-KT' : '1MOK-00083',
    '1MOK0000-84-AV' : '1MOK-00084',
    '1MOK0000-84-AV' : '1MOK-00084',
    '1MOK0088-VV' : '1MOK-00088',
    '1MOK0088-VV' : '1MOK-00088',
    '1Y1G-00136NF' : '1YAG-00136',
    'YOL-00002--DD' : '1YOL-00002',
    '2ADI-OOO60-SK' : '2ADI-00060',
    '2JA0078' : '2DJA-00078',
    'DJA-00233' : '2DJA-00233',
    'DJA-00234' : '2DJA-00234',
    'DJA-00258' : '2DJA-00258',
    'ESP-00004' : '2ESP-00004',
    'ESP-00006' : '2ESP-00006',
    '4BRH-OO510-BV' : '4BRH-00510',
    '4BRH--1260-UB' : '4BRH-01260',
    '4BR4H-01267-SN' : '4BRH-01267',
    '4BR4H-01267-SN' : '4BRH-01267',
    '4NHC=-00035-ES' : '4NHC-00035',
    '4WWUM-00033-IB' : '4WUM-00033',
    'LALL-00029-MP' : '6ALL-00029',
    'MBI-00254-FK' : '6MBI-00254',
    '6MB-00516-NS-SPU' : '6MBP-00516',
    'MBP-01654-NK' : '6MBP-01654',
    'MBP-01654-NK' : '6MBP-01654',
    'MBP-02816-PH' : '6MBP-02816',
    'MBP-03862-DA' : '6MBP-03862',
    'BRH-00133-VM' : '4BRH-00133',
    "BRH-00119-TR" : "4BRH-00119",
    "1D1R-00019-KN" : "1DAR-00019",
    "1D1R-00024-NM" : "1DAR-00024",
    "1D1R-00020-AA" : "1DAR-00020",
    "DOG-00020-AT" : "1DOG-00020",
    "DOG-00019-HB" : "1DOG-00019",
    "1DJK-00016-SPU-1" : "1DJK-00016",
    "1DJK-00014-SPU-1" : "1DJK-00014",
    "1DJK-00003-SPU-1"  : "1DJK-00003",
    "1DJK-00001-SPU-1" : "1DJK-00001",
    "1DJK-00005-SPU-1" : "1DJK-00005",
    '1MOK0000-42-MM' :'1MOK-00042',
    '1MOK0000-41-FK' :'1MOK-00041',
    '1MKL0000-40-EM' :'1MOK-00040',
    '1MOK0000-60-MM' :'1MOK-00060',
    '1MOK0000-27-ZD' :'1MOK-00027',
    '1MOK0000-59-KL' :'1MOK-00059',
    '1MOK00042-MM' :'1MOK-00042',
    '1MKL000041-FK' :'1MKL-000041',
    '1MKL000040-EM' :'1MKL-00040',
    '1DJK 00016' :'1DJK-00016',
    '1DJK 000-14' :'1DJK-00014',
    '1DJK 00003' :'1DJK-00003',
    '1DJK 00001' :'1DJK-00001',
    '1DJK 00005' :'1DJK-00005',
    "1MDM-00001MDM-00018-KO" : "1MDM-00018",
    "ESP-00008" : "2ESP-00008",
    "ESP-00009" : "2ESP-00009",
    "ESP-OOOO10" : "2ESP-00010",
    "6MBP02645-NN" : "6MBP-02645",





}

#### Define other Minor Dictionaries

In [ ]:

# List of Notes to replace with noting
notes_to_replace_with_nothing = [
    'P-047-SN',
    'P-038-NN',
    'P-059-LR',
    'P012-3-0-0',
    'P-00-OT',
    'P-030-ZA',
    'P-00-OT',
    '1MOK000-12-JF-13-ZB-14-ME',
    '1MOK000-12-JF-13-ZB-14-ME',
    '1MOK000-12-JF-13-ZB-14-ME',
    '1DJK-00491-',
    'LS 61493-PS-1SPU-1',
    "POOL44-2712GORSOU ETIENNE2718ZARA KOURAMA2719MAIGONWA MARTINE2720MAHAMAD", 
    "LS 61493-PS-1SPU-1",
    "POOL44-2712GORSOU ETIENNE2718ZARA KOURAMA2719MAIGONWA MARTINE2720MAHAMAD",
    "HAMADOU SOULEY A et B"




]

### Process Combined Files

In [ ]:
# append the two data frames together
df = pd.concat([dfe,dff])

# create a dictionary to rename the columns using partial replacement
detail_dic = {"RÉUSSITE" : 'PASS',
	"ÉCHEC" : 'FAIL',
	'SO,' : 'NA,',
	"SO" : 'NA'
}
# do partial replacement of the detail columns
special_columns = ['SPC', 'IS1081-IS6110', 'rpoB1', 'rpoB2', 'rpoB3', 'rpoB4']
df[special_columns] = df[special_columns].replace(detail_dic, regex=True)

#extract only TB samples
df = df[df['Assay']=='Xpert MTB-RIF Ultra']

# determine pool IDs that are wrongly placed in the Patient ID field

def replace_sample_id_if_match(row, id_list):
    # Convert row['Patient ID'] to string and strip any whitespace
    patient_id = str(row['Patient ID']).strip()

    # Check if the formatted patient_id is in the list
    if patient_id in id_list:
        return patient_id
    else:
        return row['Sample ID']

# Update the Sample ID with the Patient ID value if it matches a pool ID
df['Sample ID'] = df.apply(lambda row: replace_sample_id_if_match(row, patient_id_list), axis=1)

# replace values in the SAMPLE ID using the replacement dictionary

df['Sample ID'] = df['Sample ID'].replace(pool_id_dict)

# define a function to extract the pool ID from the sample ID
def extract_pool_id(sample_id):
    """
    This function extracts the potential pool ID from the sample ID.
    - It returns None for sample IDs that start with a number from 1 to 6 followed by 3 letters.
    - It extracts and returns pool IDs that start with 'P' (case-insensitive) or 'S' followed by 3 digits and a dash.
    """
    if pd.isnull(sample_id) or not isinstance(sample_id, str):
        return None
    
    # Pattern to exclude sample IDs that start with a number from 1 to 6, followed by 3 letters
    individual_pattern = r'^[1-6][A-Za-z]{3}'
    
    # If the sample_id matches the individual_pattern, return None immediately
    if re.match(individual_pattern, sample_id):
        return None

    # Regular expression to capture pool patterns starting with "P" (case insensitive) or "S" followed by 3 digits and a dash
    pool_pattern = r'^(?:[Pp].*|[Ss]\d{3,}-.*)'
    matches = re.match(pool_pattern, sample_id)
    if matches:
        return matches.group(0)
    return None

# Extract potential pool IDs and save in a new column 'extracted_pool_ids'
df['extracted_pool_ids'] = df['Sample ID'].apply(extract_pool_id)

# function to clean the extracted pool IDs
def replace_space_in_extracted_id(extracted_id):
    """
    This function cleans up the extracted_pool_ids based on specific patterns.
    """
    if pd.isnull(extracted_id):
        return extracted_id

    # Pattern that matches a string starting with P or PS followed by at least two digits and then a space
    pattern1 = r'^(P|PS)\d{2,}\s'
    # Pattern that matches a string starting with P or p or PS followed by a space and then at least two digits
    pattern2 = r'^(P|p|PS)\s\d{2,}'
    # Pattern that matches a string starting with P or PS followed by a dash and then at least two digits
    pattern3 = r'^(P|PS)-\d{2,}'

    if re.match(pattern1, extracted_id):
        extracted_id = extracted_id.replace(" ", "-", 1)
    elif re.match(pattern2, extracted_id):
        extracted_id = extracted_id.replace(" ", "", 1)
    elif re.match(pattern3, extracted_id):
        extracted_id = extracted_id.replace("-", "", 1)
    
    # Remove all occurrences of -R and -r
    extracted_id1 = extracted_id.replace("-R", "").replace("-r", "").replace("(R)","").replace("RR","")
    
    # Replace consecutive dashes with a single dash
    extracted_id2 = re.sub(r'-+', '-', extracted_id1)
    # clean the extracted id
    extracted_id3 = extracted_id2.strip()

    # replace = with dash in extracted_id3
    extracted_id4 = extracted_id3.replace("=","-")

    # replace ; with nothing in extracted_id4
    extracted_id5 = extracted_id4.replace(";", "")
    # remove PATIENT DE REDCAP
    extracted_id6 = extracted_id5.replace("PATIENT DE REDCAP", "")

    # Replace the subsequent space with nothing
    extracted_id7 = extracted_id6.replace(" ","")

    return extracted_id7


# Apply the function to the 'extracted_pool_ids' column
df['extracted_pool_ids'] = df['extracted_pool_ids'].apply(replace_space_in_extracted_id)


# define a function to remove the left over spaces and other foriegn characters
def replace_spaces_and_double_dashes(text):
    """
    Replace all spaces within a string with a dash and then replace all double dashes with a single dash.
    
    Parameters:
    - s (str): Input string
    
    Returns:
    - str: Modified string
    """
    if not isinstance(text, str):
        return text
    text1 = text.replace(" ", "-")  # Replace spaces with dashes
    text2 = text1.replace("--", "-")  # Replace double dashes with a single dash
    text3 = text2.replace("-R","") # replaece R character
    text4 = text3.replace("R","")
    text5 = text4.replace("=", "-") # Replace equal to sign with dash
    text6 = text5.replace("-2DJA-", "-")
    text7 = text6.strip() # Remove spaces 
    text8 = text7.replace("-()","")
    return text8

# Test
df['extracted_pool_ids'] = df['extracted_pool_ids'].apply(replace_spaces_and_double_dashes)


# DEFINE A FUNCTION FOR CLEANING THE NOTES FIELD
def clean_notes(notes):
    if pd.isnull(notes) or not isinstance(notes, str):
        return notes
    
    # Remove everything from "user" till the end
    notes1 = re.sub(r'user.*$', '', notes, flags=re.IGNORECASE)
    
    # Split on comma and retain the first part
    notes2 = notes1.replace(","," ")
    
    # Trim leading and trailing spaces
    notes3 = notes2.strip()
    
    # Insert a dash after the three letters for the specified pattern
    notes4 = re.sub(r'(?<=[1-6][A-Za-z]{3})(?=\d{5})', '-', notes3)
    
    # Replace space with a dash for the specified pattern
    notes5 = re.sub(r'(?<=[1-6][A-Za-z]{3})\s', '-', notes4)
    
    # Replace multiple spaces with a single space
    notes6 = re.sub(r'\s+', ' ', notes5)
    notes7 = notes6.replace("LS:" , '')
    notes8 = notes7.replace("LS " , ' ')    
    notes9 = notes8.replace(" LS " , ' ')
    notes10 = notes9.replace("ls " , ' ')   
    notes11 = notes10.replace(" ls " , ' ')
    notes12 = notes11.replace("ls:" , ' ')
    notes13 = notes12.replace("," , ' ')
    notes14 = notes13.replace("  " , ' ')
    notes15 = notes14.strip()
    notes16 = notes15.replace("." , '-')
    
    return notes16

def remove_names_and_dates(text):
    if not isinstance(text, str):
        return text

    # Find the last occurrence of a dash followed by any characters and then a space
    pattern = r'-(?:[^-]*?)(\s)([^-]*$)'
    match = re.search(pattern, text)
    if match:
        text = text[:match.start(1)].strip()  # Remove everything from the found space to the end

    return text

# Applying the function to the 'Notes' column
df['clean_notes'] = df['Notes'].apply(clean_notes)

# Re-applying the function to the 'clean_notes' column
df['clean_notes'] = df['clean_notes'].apply(remove_names_and_dates)

# Replace specified notes with NaN
df['clean_notes'] = df['clean_notes'].replace(notes_to_replace_with_nothing, np.nan)

# Replace 'extracted_pool_ids' with NaN where 'Notes' field is blank and 'extracted_pool_ids' is not null
df.loc[(df['clean_notes'] == '') & (df['extracted_pool_ids'].notnull()), 'extracted_pool_ids'] = np.nan

# replace the cleaned notes with the dictionary
df['clean_notes'] = df['clean_notes'].replace(clean_notes_replacement_dict)

# drop the rows where extracted_pool_ids are not blank but the clean notes is blank
# Condition to drop rows
condition = (df['extracted_pool_ids'].notna()) & (df['clean_notes'].isna())

# Dropping rows that meet the condition
df = df.drop(df[condition].index)

#calculate the pool size by counting the number of spaces in the clean notes field
def count_spaces_in_notes(notes, extracted_pool_ids):
    """
    Count the number of spaces in the notes field after cleaning and trimming,
    but only if extracted_pool_ids is not null.

    :param notes: The notes field (string).
    :param extracted_pool_ids: The extracted pool IDs field.
    :return: The number of spaces in the cleaned and trimmed notes field if extracted_pool_ids is not null, otherwise 0.
    """
    if pd.isnull(notes) or pd.isnull(extracted_pool_ids):
        return 0

    # Clean and trim the notes field
    cleaned_notes = notes.strip()

    # Replace double spaces with a single space
    while "  " in cleaned_notes:
        cleaned_notes = cleaned_notes.replace("  ", " ")

    # Count the number of spaces
    return cleaned_notes.count(" ")


# Apply the function to create a new column with the number of spaces
df['pool_size'] = df.apply(lambda x: count_spaces_in_notes(x['clean_notes'], x['extracted_pool_ids']), axis=1) +1

#update the pool size to 1 if the clean notes field is empty
def set_pool_size(row):
    if pd.isnull(row['clean_notes']) or pd.isnull(row['extracted_pool_ids']):
        return 1
    else:
        return row['pool_size']

# Apply the function to each row
df['pool_size'] = df.apply(set_pool_size, axis=1)

# create a column is_pool if pool size is greater than 1
df['is_pool'] = df['pool_size'] > 1


# classify results into groups
error_results = ['ERROR', 'NO RESULT', 'INVALID']

#positive result groups
positive_results = [
        'MTB DETECTED MEDIUM||RIF Resistance NOT DETECTED',
        'MTB DETECTED HIGH||RIF Resistance NOT DETECTED', 
        'MTB DETECTED LOW||RIF Resistance NOT DETECTED',
        'MTB DETECTED VERY LOW||RIF Resistance NOT DETECTED',
        'MTB DETECTED MEDIUM||RIF Resistance DETECTED',
        'MTB DETECTED LOW||RIF Resistance DETECTED',
        'MTB DETECTED HIGH||RIF Resistance DETECTED',
        '|MTB Trace DETECTED|RIF Resistance INDETERMINATE', 
        'MTB DETECTED VERY LOW||RIF Resistance DETECTED',
        'MTB DETECTED LOW||RIF Resistance INDETERMINATE',
        'MTB DETECTED HIGH||RIF Resistance INDETERMINATE',
        'MTB DETECTED VERY LOW||RIF Resistance INDETERMINATE',
        'MTB Trace DETECTED|RIF Resistance INDETERMINATE',
        'MTB DETECTED MEDIUM||RIF Resistance INDETERMINATE',
        'MTB DETECTED VERY LOW||RIF Resistance INDETERMINATE',
        'MTB DETECTED HIGH||RIF Resistance INDETERMINATE',
        'MTB DETECTED MEDIUM||RIF Resistance DETECTED',
        'MTB DETECTED LOW||RIF Resistance INDETERMINATE',
       
       
       ]

#negative result groups
negative_results = ['MTB NOT DETECTED||']

#function to classify the test results
def classify_test_result(row):
    # Check the 'is_pool' value of the current row
    if row['is_pool'] == False:
        return 'individual'

    if row['Test Result'] in error_results:
        return 'error'
    elif row['Test Result'] in positive_results:
        return 'positive'
    elif row['Test Result'] in negative_results:
        return 'negative'
    else:
        return 'unknown'

# Applying the function to the DataFrame
df['pool_result'] = df.apply(classify_test_result, axis=1)

# segment the data frames by pools
df_pool = df[df['is_pool']==True]
df_individual = df[df['is_pool']==False]

# make a copy of the clean notes field
df_pool['clean_notes_exploded'] = df_pool['clean_notes']

# define a function to explode the dataframe using the Clean Notes field
def explode_dataframe(df_pool):
    # Split the 'clean_notes' column by space and explode it
    df_exploded = df_pool.assign(clean_notes=df_pool['clean_notes'].str.split(' ')).explode('clean_notes')
    return df_exploded
df_pool = explode_dataframe(df_pool)

df = pd.concat([df_pool, df_individual])


# define a function to further clean the Notes field to enable the extraction of the QR Code
def further_clean_notes(text):
    if not isinstance(text, str):
        return text

    # Handle the format without a dash
    pattern1 = r'([1-6][A-Za-z]{3})(\d{1,4})(?!\d|-)'
    replacement1 = lambda m: m.group(1) + '-' + m.group(2).zfill(5)
    text = re.sub(pattern1, replacement1, text)

    # Handle the format with a dash but less than 5 digits after the dash
    pattern2 = r'([1-6][A-Za-z]{3}-)(\d{1,4})(?!\d)'
    replacement2 = lambda m: m.group(1) + m.group(2).zfill(5)
    text2 = re.sub(pattern2, replacement2, text)

    # Handle the format with 6 digits after the dash, where the first digit is 0
    pattern3 = r'([1-6][A-Za-z]{3}-)0(\d{5})'
    replacement3 = lambda m: m.group(1) + m.group(2)
    text3 = re.sub(pattern3, replacement3, text2)

    return text3

#clean the clean notes fied to extract qr codes
df['clean_notes'] = df['clean_notes'].apply(further_clean_notes)
df['clean_notes'] = df['clean_notes'].apply(further_clean_notes)
df['Sample ID'] = df['Sample ID'].apply(further_clean_notes)
df['Patient ID'] = df['Patient ID'].apply(further_clean_notes)

df['clean_notes'] = df['clean_notes'].str.upper()
df['Sample ID'] = df['Sample ID'].str.upper()
df['Patient ID'] = df['Patient ID'].str.upper()

# Replace the wrongly typed qr code with the qr_code_replacement_dict
df['clean_notes'] = df['clean_notes'].replace(qr_code_replacement_dict)

# do the replacement in Sample ID field
df['Sample ID'] = df['Sample ID'].replace(qr_code_replacement_dict)

# do the replacement in Patient ID field
df['Patient ID'] = df['Patient ID'].replace(qr_code_replacement_dict)

# define a function to extract the QR Code
def extract_qr_codes(df):
    # Define the regex pattern for a QR code
    qr_pattern = r'([1-6][A-Z]{3}-\d{5})'
    
    def get_qr_code(row):
        # First attempt to extract from the 'clean_notes' column if 'is_pool' is True
        if row['is_pool']:
            match = re.search(qr_pattern, str(row['clean_notes']))
            if match:
                return match.group(0)
        
        # If not found in 'clean_notes' or 'is_pool' is False, try the 'Sample ID' column
        match = re.search(qr_pattern, str(row['Sample ID']))
        if match:
            return match.group(0)
        
        # If not found in 'Sample ID', try the 'Patient ID' column
        match = re.search(qr_pattern, str(row['Patient ID']))
        if match:
            return match.group(0)
        
        return None  # If no QR code found in any of the fields

    # Extract QR codes based on the function
    df['qr_codes'] = df.apply(get_qr_code, axis=1)
    
    return df

df = extract_qr_codes(df)


# Function to extract a QR code from a single row
def get_qr_code2(row):
    qr_pattern = r'([1-6][A-Z]{3}-\d{5})'

    # Check 'Sample ID' for a QR code
    match = re.search(qr_pattern, str(row['Sample ID']))
    if match:
        return match.group(0)
    
    # If not found, check 'Patient ID'
    match = re.search(qr_pattern, str(row['Patient ID']))
    if match:
        return match.group(0)

    # If still not found, check 'clean_notes'
    match = re.search(qr_pattern, str(row['clean_notes']))
    if match:
        return match.group(0)
    
    return None  # Return None if no QR code found

# Update 'qr_codes' where it is currently null
mask = df['qr_codes'].isnull()
df.loc[mask, 'qr_codes'] = df[mask].apply(get_qr_code2, axis=1)

# Now df contains updated 'qr_codes' values
# define a function to extract the S4A QR Code
def extract_s4a_qr_codes(row, columns_to_search):
    """
    Extract the first S4A QR code from specified columns of a DataFrame row.

    :param row: A row of the DataFrame.
    :param columns_to_search: List of column names to search for the QR code.
    :return: The first extracted S4A QR code or NaN if none found.
    """
    pattern = r'SC[1-5]-\d{4}'

    for column in columns_to_search:
        if not isinstance(row[column], str):
            continue

        match = re.search(pattern, row[column])
        if match:
            return match.group(0)

    return np.nan

# Define columns to search for QR codes
columns_to_search = ['clean_notes', 'Patient ID', 'Sample ID']

# Apply the function to create a new column with the extracted S4A QR codes
df['s4a'] = df.apply(extract_s4a_qr_codes, args=(columns_to_search,), axis=1)

# Define a function to determine the project based on s4a and qr_codes
def determine_project(row):
    if pd.notnull(row['s4a']):
        return 'S4A'
    elif pd.notnull(row['qr_codes']):
        return 'Inspire'
    else:
        return 'Other'

# Apply the function to create a new column 'project'
df['project'] = df.apply(determine_project, axis=1)

# Update 'qr_codes' to the value of 's4a' if 'qr_codes' is null
df['qr_codes'] = df.apply(lambda row: row['s4a'] if pd.isnull(row['qr_codes']) else row['qr_codes'], axis=1)

# define a function to update the qr code in place
def update_qr_codes_inplace(df):
    """
    Update the qr_codes column in the DataFrame in place.
    If is_pool is True and qr_codes is null, set qr_codes to clean_notes.
    Otherwise, set qr_codes to Sample ID where qr_codes is null.

    Parameters:
    df (pandas.DataFrame): The DataFrame to be updated.
    """
    # Compute the values to be assigned
    values_to_assign = np.where(
        df['is_pool'] & df['qr_codes'].isnull(), 
        df['clean_notes'], 
        df['Sample ID']
    )

    # Filter the values to match the length of rows where 'qr_codes' is null
    values_to_assign = values_to_assign[df['qr_codes'].isnull()]

    # Assign the computed values to 'qr_codes' where 'qr_codes' is null
    df.loc[df['qr_codes'].isnull(), 'qr_codes'] = values_to_assign

update_qr_codes_inplace(df)

# Merge the xpert files with the main dataframe
df = df.merge(xpert_files[['file', 'lab', 'start_date', 'end_date', 'date_created']], on = 'file', how = 'left')

# Convert 'date_created' to datetime format
df['date_created'] = pd.to_datetime(df['date_created'])

# Sort by 'Cartridge S/N' and 'date_created' in descending order
df = df.sort_values(by=['Cartridge S/N', 'date_created'], ascending=[True, False])

# Group by 'Cartridge S/N' and keep only the rows with the latest 'date_created' for each group
df = df[df['date_created'] == df.groupby('Cartridge S/N')['date_created'].transform('max')]

# sort the dataframe by the pool result, pool size, and qr codes
df.sort_values(by =['is_pool','Cartridge S/N', 'file'], ascending=[False, True, True], inplace = True)
df = df.drop_duplicates(subset = ['Cartridge S/N','qr_codes'])
df.dropna(subset = ['qr_codes'], inplace = True)

# Define the desired categories order
categories_order = ['positive', 'negative', 'individual', 'error']

# Convert the 'pool_result' column to categorical with the specified order
df['pool_result'] = pd.Categorical(df['pool_result'], categories=categories_order, ordered=True)

# Convert the 'pool_result' column to categorical with the specified order
df['pool_result'] = pd.Categorical(df['pool_result'], categories=categories_order, ordered=True)

#drop duplicates
df.drop_duplicates(subset = ['Cartridge S/N', 'qr_codes'], inplace = True)

df.sort_values(by=['date_created','is_pool', 'pool_result', 'qr_codes',], ascending=[False, False, True, True,], inplace=True)

#determine the duplicate number for each qr code
df['dup_no'] = df.groupby('qr_codes').cumcount() + 1
# Sort the dataframe
df.sort_values(by=['pool_result', 'qr_codes', 'dup_no'], ascending=[True, True, True], inplace=True)

#Write a function to get the individual results
def get_individual_values(row, df):
    # Check if the row is a positive pooled sample
    if row['is_pool'] and row['pool_result'] == 'positive':
        # Create a subset DataFrame for individual tests matching the QR code of the pool
        subset_df = df[(df['qr_codes'] == row['qr_codes']) & (~df['is_pool'])]
        # If individual tests are found, return the Test Result of the first one
        if not subset_df.empty:
            return subset_df['Test Result'].iloc[0]
        # If no individual tests are found, return "No Individual Results Yet"
        else:
            return "No Ind Test Yet"
    # For all other cases, return the Test Result of the row
    return row['Test Result']


# Apply the function to the 'Test Result' column of df_valid
df['individual_results'] = df.apply(lambda row: get_individual_values(row, df), axis=1)

# function to get the individual cartridge SN
def get_individual_cartridge_sn(row):
    # If the row represents a pool and the result is positive
    if row['is_pool'] and row['pool_result'] == 'positive':
        # Filter df to find the corresponding row where 'is_pool' is False
        subset_df = df[(df['qr_codes'] == row['qr_codes']) & (~df['is_pool'])]

        # If there's a match, return the 'Cartridge S/N' value from the subset
        if not subset_df.empty:
            return subset_df['Cartridge S/N'].iloc[0]
        # If no individual test is found, return "No Individual Results Yet"
        else:
            return None
    
    # For all other cases (including pools that are not positive), return None
    return None

df['individual_cartridge_sn'] = df.apply(lambda row: get_individual_cartridge_sn(row), axis=1)

# Define the categories based on individual results
def categorize_result(individual_result):
    if individual_result in positive_results:
        return 'positive'
    elif individual_result in negative_results:
        return 'negative'
    elif individual_result in error_results:
        return 'invalid'
    else:
        return 'unknown'

df['indiv_result_category'] = df['individual_results'].apply(categorize_result)


#create the final pool size for individuals where the pool was positive
def update_pool_size(df):
    # Create a new column 'final_pool_size' with the same values as 'pool_size' initially
    df['final_pool_size'] = df['pool_size']
    
    # Find the unique xpert_ids with more than one occurrence
    xpert_ids = df['qr_codes'].value_counts()
    xpert_ids = xpert_ids[xpert_ids > 1].index.tolist()
    
    # Iterate through each unique xpert_id
    for xpert_id in xpert_ids:
        # Check if xpert_id has both is_pool True and False entries
        if df[(df['qr_codes'] == xpert_id) & (df['is_pool'])].empty or df[(df['qr_codes'] == xpert_id) & (~df['is_pool'])].empty:
            continue  # Skip ids that don't have both True and False entries
        
        # Get the pool size where is_pool is True
        true_pool_size = df[(df['qr_codes'] == xpert_id) & (df['is_pool'])]['pool_size'].iloc[0]
        
        # Update the 'final_pool_size' where is_pool is False for the same xpert_id
        df.loc[(df['qr_codes'] == xpert_id) & (~df['is_pool']), 'final_pool_size'] = true_pool_size
        
    return df

# Now call the function to update the DataFrame
df = update_pool_size(df) 

# if final pool size  = 1, then test type = individual else pooled
df['test_type'] = np.where(df['final_pool_size']==1, 'individual', 'pooled')

# Define the QR code pattern
qr_pattern = r'([1-6][A-Z]{3}-\d{5})'

# Function to extract the first four characters if qr_code matches the pattern
def extract_site(qr_code):
    if re.match(qr_pattern, qr_code):
        return qr_code[:4]
    else:
        return None  # Or return '' if you prefer an empty string for non-matches

# Create the 'site' column based on 'qr_codes'
df['site'] = df['qr_codes'].apply(extract_site)

# determine if someone has TB
df['has_tb'] = df['indiv_result_category'] == 'positive'

# where site is not null, xpert_id = qr_codes
df['xpert_id'] = np.where(df['site'].notnull(), df['qr_codes'], np.nan)

# if both extracted_pool_ids and Notes are notnull, then pool_id = extracted_pool_ids
df['pool_id'] = np.where(df['extracted_pool_ids'].notnull() & df['Notes'].notnull(), df['extracted_pool_ids'], np.nan)


# convert datetime columns to datetime
df['Expiration Date'] = df['Expiration Date'].apply(fix_dates)
df['Start Time'] = df['Start Time'].apply(fix_dates)
df['End Time'] = df['End Time'].apply(fix_dates)
# Define the date for comparison
date_threshold = pd.Timestamp('2023-06-27')
current_date = pd.Timestamp.today()

# Create the 'date' column based on the conditions
df['date'] = np.where(
    df['End Time'] < date_threshold, df['start_date'],
    np.where(df['End Time'] > current_date, df['end_date'], df['End Time']))

#fill missing values in the date column with the enddate value
df['date'] = df['date'].fillna(df['end_date'])

def classify_pool(group):
    """
    Classify the pool based on individual test results within the group.
    """
    if group['pool_result'].iloc[0] == 'error':
        return 'Error'
    elif group['pool_result'].iloc[0] == 'negative':
        return 'Negative'
    elif group['pool_result'].iloc[0] == 'positive':
        if all(group['indiv_result_category'] == 'negative'):
            return 'False Positive'
        elif any(group['indiv_result_category'] == 'positive'):
            return 'True Positive'
        elif all((group['indiv_result_category'] == 'negative') | (group['indiv_result_category'] == 'unknown')):
            return 'Incomplete'
    return 'Unknown'

# Filter the DataFrame for pooled samples and apply the classification function to each pooled group
pooled_df = df[df['is_pool']]
pooled_classifications = pooled_df.groupby('Cartridge S/N').apply(classify_pool)

# Map the classifications back to the original DataFrame for pooled samples
df.loc[df['is_pool'], 'pool_classification'] = df['Cartridge S/N'].map(pooled_classifications)

# Set pool_classification to NaN where final_pool_size is 1
df.loc[df['final_pool_size'] == 1, 'pool_classification'] = np.nan

#drop unnecessary columns
df.drop(columns= ['rpoB1 melt', 'rpoB2 melt', 'rpoB3 melt', 'rpoB4 melt', 'rpoB1 Mut melt', 'rpoB2 Mut melt', 'rpoB3 Mut melt',
       'rpoB4 Mut melt A', 'rpoB4 Mut melt B', 'Patient ID 2', 'Assay', 'Assay Version', 'Assay Type', 'Test Type',
       'clean_notes', 'Module Name', 'start_date' , 's4a', 'start_date', 'end_date', 'date_created', 'Sample Type', 'clean_notes',], inplace=True)

In [ ]:
df

: 